In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import utils
from darts import TimeSeries
import numpy as np
import pandas as pd
from darts.utils import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from darts.metrics import rmse, mape, r2_score
from statsmodels.tsa.stattools import grangercausalitytests

c:\Users\joren\OneDrive\Documents\GitHub\cdpq-ia\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = utils.load_data()
ts, covars_diff, covars_nodiff = utils.df2ts(df)

In [413]:
covars_diff_pipeline, covars_diff_scaled = utils.scale_ts(covars_diff, should_diff=True)
covars_nodiff_pipeline, covars_nodiff_scaled = utils.scale_ts(
    covars_nodiff, should_diff=False
)
pipeline, ts_scaled = utils.scale_ts(ts, should_diff=True)

train_ts_scaled, val_ts_scaled = ts_scaled.split_after(pd.Timestamp("2016-01-01"))
train_ts, val_ts = ts.split_after(pd.Timestamp("2016-01-01"))

covariates_scaled = covars_diff_scaled.stack(covars_nodiff_scaled)
train_covariates_scaled, val_covariates_scaled = covariates_scaled.split_after(
    pd.Timestamp("2016-01-01")
)

In [ ]:
#Faire de de nouveau les test de Granger, mais cette-fois avec les données de la FRED dans "data_concours"
data1 = pd.read_csv("data/data_concours.csv", parse_dates=["DATE"]).set_index("DATE")

data1 = data1.rolling(window=12).mean()
data1 = data1.asfreq(freq="ME", method="ffill")
data1 = data1[["SKEW_INDEX_YOY", "US_TB_YIELD_10YRS"]]  


ts_df1 = TimeSeries.from_dataframe(data1, value_cols=["SKEW_INDEX_YOY", "US_TB_YIELD_10YRS"])
pipeline, ts_scaled = utils.scale_ts(ts_df1, should_diff=True)
scaled_df = ts_scaled.pd_dataframe()

gc_res = grangercausalitytests(scaled_df, maxlag=12, verbose = False)

# Extraire uniquement les informations pertinentes
significant_lags = []
threshold = 0.05  # Niveau de signification (5%)

for lag, result in gc_res.items():
    p_value = result[0]['ssr_ftest'][1]  # Récupère la p-value du F-test
    if p_value < threshold:
        significant_lags.append((lag, p_value))

# Afficher les résultats significatifs
if significant_lags:
    print("Lags avec p-values significatives (p < 0.05) :")
    for lag, p_value in significant_lags:
        print(f"Lag {lag}: p-value = {p_value:.4f}")
else:
    print("Aucun lag significatif trouvé (p < 0.05).")


print(gc_res)


Aucun lag significatif trouvé (p < 0.05).
{1: ({'ssr_ftest': (0.23637677303861998, 0.6269781186986099, 746.0, 1), 'ssr_chi2test': (0.23732734987389595, 0.6261431425944903, 1), 'lrtest': (0.2372897581676625, 0.6261704836017428, 1), 'params_ftest': (0.2363767730388718, 0.6269781186984288, 746.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1871D60D0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F187571850>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (0.1659375147359022, 0.8471305481157692, 743.0, 2), 'ssr_chi2test': (0.3341083742192594, 0.8461537594683815, 2), 'lrtest': (0.3340337785130032, 0.8461853197755881, 2), 'params_ftest': (0.16593751473582602, 0.847130548115804, 743.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F187572C90>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F187571B90>, array([[0., 0., 1., 0., 0.],
       [0., 0

c:\Users\joren\OneDrive\Documents\GitHub\cdpq-ia\venv\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [401]:
#Faire de de nouveau les test de Granger, mais cette-fois avec les données de la FRED dans "Variables_Us"
data2 = pd.read_csv("data/excel_jorge/Variables_US.csv", parse_dates=["DATE"]).set_index("DATE")

data2 = data2.rolling(window=12).mean()
data2 = data2.asfreq(freq="ME", method="ffill")
data3 = data1[["US_TB_YIELD_10YRS"]]  

data4 = data2.join(data3, how='inner')

data4 = data4[["EXPINF10YR", "US_TB_YIELD_10YRS"]]

ts_df1 = TimeSeries.from_dataframe(data4, value_cols=["EXPINF10YR", "US_TB_YIELD_10YRS"])
pipeline, ts_scaled = utils.scale_ts(ts_df1, should_diff=True)
scaled_df = ts_scaled.pd_dataframe()

gc_res = grangercausalitytests(scaled_df, maxlag=12, verbose = False)

# Extraire uniquement les informations pertinentes
significant_lags = []
threshold = 0.05  # Niveau de signification (5%)

for lag, result in gc_res.items():
    p_value = result[0]['ssr_ftest'][1]  # Récupère la p-value du F-test
    if p_value < threshold:
        significant_lags.append((lag, p_value))

# Afficher les résultats significatifs
if significant_lags:
    print("Lags avec p-values significatives (p < 0.05) :")
    for lag, p_value in significant_lags:
        print(f"Lag {lag}: p-value = {p_value:.4f}")
else:
    print("Aucun lag significatif trouvé (p < 0.05).")


print(gc_res)

Lags avec p-values significatives (p < 0.05) :
Lag 1: p-value = 0.0000
Lag 2: p-value = 0.0000
Lag 3: p-value = 0.0000
Lag 4: p-value = 0.0000
Lag 5: p-value = 0.0000
Lag 6: p-value = 0.0000
Lag 7: p-value = 0.0000
Lag 8: p-value = 0.0000
Lag 9: p-value = 0.0000
Lag 10: p-value = 0.0000
Lag 11: p-value = 0.0000
Lag 12: p-value = 0.0000
{1: ({'ssr_ftest': (97.87363027656941, 9.099533227879909e-22, 746.0, 1), 'ssr_chi2test': (98.2672239640087, 3.6556684506222445e-23, 1), 'lrtest': (92.33458986010601, 7.319374276427091e-22, 1), 'params_ftest': (97.87363027656964, 9.099533227879068e-22, 746.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1875B9F50>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1875AF950>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (43.73656728064161, 1.103795074370088e-18, 743.0, 2), 'ssr_chi2test': (88.06178284231473, 7.544436760394869e-20, 2), 'lrtest': (83.25205240512355, 8.356925232810977e-

c:\Users\joren\OneDrive\Documents\GitHub\cdpq-ia\venv\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [411]:
#Faire de de nouveau les test de Granger, mais cette-fois avec les données de la FRED dans "Variables_Us_Weekly"
data5 = pd.read_csv("data/excel_jorge/Variables_US_Weekly.csv", parse_dates=["DATE"]).set_index("DATE")
data5 = data5.resample("ME").mean()

#data5 = data5.rolling(window=12).mean()
#data5 = data5.asfreq(freq="ME", method="ffill")
data3 = data1[["US_TB_YIELD_10YRS"]]  

data6 = data5.join(data3, how='inner')

data6 = data6[["TOTLL", "US_TB_YIELD_10YRS"]]

ts_df1 = TimeSeries.from_dataframe(data6, value_cols=["TOTLL", "US_TB_YIELD_10YRS"])
pipeline, ts_scaled = utils.scale_ts(ts_df1, should_diff=True)
scaled_df = ts_scaled.pd_dataframe()

gc_res = grangercausalitytests(scaled_df, maxlag=12, verbose = False)

# Extraire uniquement les informations pertinentes
significant_lags = []
threshold = 0.05  # Niveau de signification (5%)

for lag, result in gc_res.items():
    p_value = result[0]['ssr_ftest'][1]  # Récupère la p-value du F-test
    if p_value < threshold:
        significant_lags.append((lag, p_value))

# Afficher les résultats significatifs
if significant_lags:
    print("Lags avec p-values significatives (p < 0.05) :")
    for lag, p_value in significant_lags:
        print(f"Lag {lag}: p-value = {p_value:.4f}")
else:
    print("Aucun lag significatif trouvé (p < 0.05).")


print(gc_res)


Aucun lag significatif trouvé (p < 0.05).
{1: ({'ssr_ftest': (0.5368564300049062, 0.4640180651783363, 614.0, 1), 'ssr_chi2test': (0.5394795070244741, 0.46264851567556187, 1), 'lrtest': (0.5392437950295061, 0.46274629135138645, 1), 'params_ftest': (0.5368564300048136, 0.46401806517839317, 614.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1875B9C90>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F18749C7D0>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (0.5518960833537055, 0.5761436981454199, 611.0, 2), 'ssr_chi2test': (1.112824835829403, 0.5732620035962642, 2), 'lrtest': (1.111820866930202, 0.5735498444473877, 2), 'params_ftest': (0.5518960833536812, 0.5761436981454394, 611.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1874ADF90>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001F1874AC6D0>, array([[0., 0., 1., 0., 0.],
       [0., 0.

c:\Users\joren\OneDrive\Documents\GitHub\cdpq-ia\venv\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [407]:
data5

,TLAACBW027SBOG,TLBACBW027SBOG,TOTBKCR,TOTLL
DATE,,,,
1973-01-31,702.838420,627.897780,566.598200,405.165000
1973-02-28,710.763225,629.659800,574.254825,415.619575
1973-03-31,719.691500,634.351675,583.669400,425.590400
1973-04-30,726.946225,644.099875,589.238800,430.881050
1973-05-31,736.497460,650.040580,595.380660,438.160660
...,...,...,...,...
2024-07-31,23472.080560,21205.802300,17725.778780,12448.712020
2024-08-31,23507.481050,21213.634400,17797.784675,12470.151525
2024-09-30,23468.246625,21155.726350,17841.928100,12480.925925
